In [ ]:
import pandas
import os
from skyfield.api import EarthSatellite, load, wgs84


# 所有的遮挡物数据csv放在路径下，共29458。提供TLE数据
trash_path = "/root/wei/TLE/all_by_id_latest/"
# 输出文件存储路径
save_path = "/root/oyl/003/data/trash_object/"
trash_file_list = os.listdir(trash_path)
# print(len(trash_file_list))

# bookmark 设定你需要的时间点
select_t = "2022-12-27 00:00:00"

for trash_file_name in trash_file_list:
    # 传播后数据存储 一分钟一行
    propagation_list = []
    trash_pd = pandas.read_csv(trash_path+trash_file_name)
    # print(trash_pd_0.shape)
    # print(trash_pd_0.columns)
    # print(trash_pd_0["EPOCH"])
    data = trash_pd
    # 筛选出距离目标时间点前最近的TLE
    temp=data[data["EPOCH"]<=select_t]
    if len(temp) != 0:
        lastest = temp.iloc[-1]
        print(lastest)
    else:
        raise Exception

    ts = load.timescale()
    satellite = EarthSatellite(lastest['TLE_LINE1'], lastest['TLE_LINE2'], lastest['TLE_LINE0'])
    # You can instead use ts.now() for the current time
    # t_init = ts.utc(2022, 12, 27, 0, 0, 0)
    # 传播时间点数 每分钟一个点 这里是60个点 bookmark
    t_lens = 60
    for i in range(t_lens):
        # bookmark
        t = ts.utc(2022, 12, 27, 0, 0 + i, 0)
        geocentric = satellite.at(t)
        # print(geocentric.position.km)
        # [-5021.82658191   742.71506112  3831.57403957]
        propagation_list.append(geocentric.position.km)
    propagation_pd = pandas.DataFrame(propagation_list)
    propagation_pd.columns = ['x', 'y', 'z']
    propagation_pd.to_csv(save_path + "Propagation" + trash_file_name)

    break